## simulation user lifetimes 

In [1]:
import sys
import pandas as pd
import time
import datetime

In [2]:
## warnings handler
import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
metrics_db = application.new_db_engine('nanohub_metrics')

salesforce = application.new_salesforce_engine()
db_s = salesforce


``````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````
```````````````````````````````$$$$$``````````````````````````````````````````````````````````````````````````````````````````````````````````````````
`````````````````````````````$$$$`$$$$````````````````````````````````````````````````````````````````````````````````````````````````````````````````
`````````````````$$$$$```````$$`````$$$```````````````````````````````````````````````````````````````````````````````````````````````````````````````
````````````````$$```$$``````$$$$``$$$````````````````````````````````````````````````````````````````````````````````````````````````````````````````
````````````````$$$$$$$```````$$$$$$``````````````````````````````````````````````````````````````````````````````````````````````````````````````````
``````$$$$````````$$$$$```````$$$``````$$$$$$````````````````````````````````````````````````

2021-08-06 14:08:30,214| WARNING | Could not read SSH configuration file: ~/.ssh/config


2021-08-06 14:08:30,214 - [WARNING] sshtunnel.SSHTunnelForwarder [sshtunnel._read_ssh_config:1032]: Could not read SSH configuration file: ~/.ssh/config
2021-08-06 14:08:30,214 - [WARNING] sshtunnel.SSHTunnelForwarder [sshtunnel._read_ssh_config:1032]: Could not read SSH configuration file: ~/.ssh/config


2021-08-06 14:08:30,221| INFO    | 0 keys loaded from agent


2021-08-06 14:08:30,221 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_agent_keys:1060]: 0 keys loaded from agent
2021-08-06 14:08:30,221 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_agent_keys:1060]: 0 keys loaded from agent


...

2021-08-06 14:08:31,810 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_keys:1117]: 0 key(s) loaded
2021-08-06 14:08:31,810 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_keys:1117]: 0 key(s) loaded


2021-08-06 14:08:31,815| INFO    | Connecting to gateway: db2.nanohub.org:22 as user 'wang2506'


2021-08-06 14:08:31,815 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.__init__:978]: Connecting to gateway: db2.nanohub.org:22 as user 'wang2506'
2021-08-06 14:08:31,815 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.__init__:978]: Connecting to gateway: db2.nanohub.org:22 as user 'wang2506'
2021-08-06 14:08:31,822 - [INFO] nanoHUB.containers.dataaccess [connection.get_connection_for:78]: Started SSH Tunnel with db2.nanohub.org


2021-08-06 14:08:32,070| INFO    | Opening tunnel: 0.0.0.0:38981 <> 127.0.0.1:3306


2021-08-06 14:08:32,070 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel._serve_forever_wrapper:1433]: Opening tunnel: 0.0.0.0:38981 <> 127.0.0.1:3306
2021-08-06 14:08:32,070 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel._serve_forever_wrapper:1433]: Opening tunnel: 0.0.0.0:38981 <> 127.0.0.1:3306
Obtained Salesforce access token ...... True


In [4]:
## read in data
import os
APP_DIR =  os.getenv('APP_DIR')
cwd = APP_DIR + '/nanoHUB/pipeline/salesforce'
clusters_df = pd.read_csv(cwd+'/manual_data_filters/clustered_users.csv')
selfs_df = pd.read_csv(cwd+'/manual_data_filters/self_study_users.csv')
researchers_df = pd.read_csv(cwd+'/manual_data_filters/researchers.csv')


In [5]:
clusters_df.head(5)

,Unnamed: 0,username
0,10813,0.yao.yuan
1,10817,008dilip
2,10821,01024461660a
3,10826,01luiyi
4,10828,0214moon


In [6]:
selfs_df.head(5)

,Unnamed: 0,username
0,0,-100005
1,1,-100024
2,2,-100026
3,3,-100032
4,4,-10006


In [7]:
researchers_df.head(5)

,Unnamed: 0,usernames
0,0,hspwong
1,1,onayfeh
2,2,accelrys
3,3,himadri
4,4,fossum


In [75]:
sql_query = "select distinct user, min(datetime), max(datetime) from toolstart group by user"
#"select min(datetime), max(datetime), user from toolstart group by user"
lifetimes_df = pd.read_sql_query(sql_query,metrics_db)

KeyboardInterrupt: 

In [10]:
lifetimes_df

,user,min(datetime),max(datetime)
0,,0000-00-00 00:00:00,2013-05-28 15:46:14
1,0.yao.yuan,2017-03-08 10:15:23,2017-03-09 05:09:22
2,008dilip,2016-09-28 20:27:13,2016-10-12 23:50:56
3,00ff,2013-11-19 00:53:23,2013-11-19 00:58:56
4,00thamizharasi00,2017-03-20 09:46:36,2017-03-20 09:46:36
...,...,...,...
154856,zzzwmhq,2021-07-05 13:19:19,2021-07-05 13:27:28
154857,z_depth,2002-07-23 02:52:49,2002-07-23 03:04:24
154858,Z_E_U_S,2006-10-16 20:25:49,2006-10-16 20:27:53
154859,z_henry35,2013-02-11 14:42:41,2013-02-19 01:53:39


In [27]:
# remove garbage usernames
temp = lifetimes_df[lifetimes_df['user'] != '']

In [28]:
temp

,user,min(datetime),max(datetime)
1,0.yao.yuan,2017-03-08 10:15:23,2017-03-09 05:09:22
2,008dilip,2016-09-28 20:27:13,2016-10-12 23:50:56
3,00ff,2013-11-19 00:53:23,2013-11-19 00:58:56
4,00thamizharasi00,2017-03-20 09:46:36,2017-03-20 09:46:36
5,01024461660a,2020-09-23 18:14:20,2020-12-11 02:58:30
...,...,...,...
154856,zzzwmhq,2021-07-05 13:19:19,2021-07-05 13:27:28
154857,z_depth,2002-07-23 02:52:49,2002-07-23 03:04:24
154858,Z_E_U_S,2006-10-16 20:25:49,2006-10-16 20:27:53
154859,z_henry35,2013-02-11 14:42:41,2013-02-19 01:53:39


In [29]:
lifetimes_df['min(datetime)'].to_numpy()[0]

'0000-00-00 00:00:00'

In [30]:
lifetimes_df['min(datetime)'].to_list()[0] == '0000-00-00 00:00:00'

True

In [37]:
# replace 0000-00-00 with 2000-01-01
temp2 = temp['min(datetime)'].to_list()
for i,j in enumerate(temp2):
    if type(j) == str:
        if j == '0000-00-00 00:00:00':
            temp2[i] = '2000-01-01 00:10:10'


In [38]:
len(temp2)

154860

In [40]:
temp2[0] == '0000-00-00 00:00:00'

False

In [41]:
temp['min(datetime)'] = temp2 #temp2[1:]
temp['min(datetime)'] = pd.to_datetime(temp['min(datetime)'])
temp['max(datetime)'] = pd.to_datetime(temp['max(datetime)'])

In [ ]:
## calculate lifetimes
lifetime_num = []
min_dates = temp['min(datetime)'].to_list()
for i,j in enumerate(temp['max(datetime)'].to_list()):
    temp_time = j-min_dates[i]
    lifetime_num.append(temp_time.days)

In [43]:
lifetime_num[:5]

[0, 14, 0, 0, 78]

In [44]:
temp['lifetimes'] = lifetime_num

In [45]:
display(temp.head(5))
display(temp.tail(5))

,user,min(datetime),max(datetime),lifetimes
1,0.yao.yuan,2017-03-08 10:15:23,2017-03-09 05:09:22,0
2,008dilip,2016-09-28 20:27:13,2016-10-12 23:50:56,14
3,00ff,2013-11-19 00:53:23,2013-11-19 00:58:56,0
4,00thamizharasi00,2017-03-20 09:46:36,2017-03-20 09:46:36,0
5,01024461660a,2020-09-23 18:14:20,2020-12-11 02:58:30,78


,user,min(datetime),max(datetime),lifetimes
154856,zzzwmhq,2021-07-05 13:19:19,2021-07-05 13:27:28,0
154857,z_depth,2002-07-23 02:52:49,2002-07-23 03:04:24,0
154858,Z_E_U_S,2006-10-16 20:25:49,2006-10-16 20:27:53,0
154859,z_henry35,2013-02-11 14:42:41,2013-02-19 01:53:39,7
154860,z_sath,2008-07-09 22:51:51,2008-07-23 09:39:34,13


In [50]:
temp['max(datetime)'][temp['max(datetime)'] >= '2021-07-01']

126      2021-07-19 06:09:53
165      2021-07-01 14:03:30
187      2021-07-17 09:05:27
195      2021-07-05 03:03:49
349      2021-08-01 03:24:25
                 ...        
154642   2021-07-19 07:24:43
154747   2021-07-24 03:28:53
154789   2021-07-30 06:09:29
154818   2021-07-25 08:57:28
154856   2021-07-05 13:27:28
Name: max(datetime), Length: 2225, dtype: datetime64[ns]

In [51]:
from copy import deepcopy

In [52]:
send_df = deepcopy(temp)

In [53]:
send_df = send_df.rename(columns={'user':'nanoHUB_username__c','lifetimes':'sim_lifetimes__c',\
                    'min(datetime)':'earliest_sim__c','max(datetime)':'latest_sim__c'})

In [54]:
send_df

,nanoHUB_username__c,earliest_sim__c,latest_sim__c,sim_lifetimes__c
1,0.yao.yuan,2017-03-08 10:15:23,2017-03-09 05:09:22,0
2,008dilip,2016-09-28 20:27:13,2016-10-12 23:50:56,14
3,00ff,2013-11-19 00:53:23,2013-11-19 00:58:56,0
4,00thamizharasi00,2017-03-20 09:46:36,2017-03-20 09:46:36,0
5,01024461660a,2020-09-23 18:14:20,2020-12-11 02:58:30,78
...,...,...,...,...
154856,zzzwmhq,2021-07-05 13:19:19,2021-07-05 13:27:28,0
154857,z_depth,2002-07-23 02:52:49,2002-07-23 03:04:24,0
154858,Z_E_U_S,2006-10-16 20:25:49,2006-10-16 20:27:53,0
154859,z_henry35,2013-02-11 14:42:41,2013-02-19 01:53:39,7


In [57]:
print(send_df['nanoHUB_username__c'].to_list().index('drjohnbegg'))
print(send_df.iloc[37081,:]) ## a sanity check

37081
nanoHUB_username__c             drjohnbegg
earliest_sim__c        2020-02-10 01:55:57
latest_sim__c          2021-08-01 09:56:56
sim_lifetimes__c                       538
Name: 37082, dtype: object


In [58]:
## send to salesforce

In [60]:
db_s.external_id = 'nanoHUB_username__c' #external_id
db_s.object_id = 'Contact' #object_id

In [61]:
db_s.send_data(send_df)

[Success] Bulk job creation successful. Job ID = 7505w00000YcuUYAAZ
hello
[Success] CSV upload successful. Job ID = 7505w00000YcuUYAAZ
[Success] Closing job successful. Job ID = 7505w00000YcuUYAAZ


In [69]:
db_s.check_bulk_status()

{'id': '7505w00000YcuUYAAZ',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-08-06T15:51:27.000+0000',
 'systemModstamp': '2021-08-06T15:57:57.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_username__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 154860,
 'numberRecordsFailed': 730,
 'retries': 0,
 'totalProcessingTime': 1656039,
 'apiActiveProcessingTime': 1477641,
 'apexProcessingTime': 12488021}

In [70]:
import pprint
pprint.pprint(db_s.check_bulk_failed_results())

('"sf__Id","sf__Error",earliest_sim__c,latest_sim__c,nanoHUB_username__c,sim_lifetimes__c\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","2017-04-26","2017-04-26","1368872691zhangheng","0.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","2004-08-26","2004-09-26","30701144","31.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","2004-07-26","2004-09-19","3418303","54.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","2006-04-04","2006-04-05","4330","1.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","2000-02-28","2000-02-28","595nta","0.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","2017-06-07","2017-06-07","6r1d","0.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","2005-03-07","2005-08-13","